In [1]:
import findspark
findspark.init('/home/purvil/spark-2.4.3-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Aggregation').getOrCreate()

In [4]:
person = spark.createDataFrame([
    (0, "Purvil Dave", 0, [100]),
    (1, "Bhavika Joshi", 1, [500,250,100]),
    (2, "Japan Dave", 1, [250,100])
]).toDF("id", "name", "graduate_program", "spark_status")

In [5]:
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D", "EECS", "UC Berkeley")
]).toDF("id", "degree", "department", "school")

In [6]:
sparkStatus = spark.createDataFrame([
    (500, "Vice president"), 
    (250, "Member"),
    (100, "Contributor")
]).toDF("id", "status")

In [7]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

### Inner Join
* Keep rows with keys that exist in the left and right datasets
* Evaluate key in both dataframe or tables and incluse only the rows that evaluate to true

In [10]:
joinExpression = person["graduate_program"] == graduateProgram['id']

In [12]:
joinExpression

Column<b'(graduate_program = id)'>

In [18]:
person.join(graduateProgram, joinExpression, "inner").show()

+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|  Purvil Dave|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|Bhavika Joshi|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|  2|   Japan Dave|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+



In [17]:
spark.sql("""
    SELECT * 
    FROM person
    INNER JOIN graduateProgram
    ON person.graduate_program = graduateProgram.id
""").show()

+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|  Purvil Dave|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|Bhavika Joshi|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|  2|   Japan Dave|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+



### Outer Join
* Keep rows with keys in either the left or right datasets.
* Evaluates keys in both dataframes or tables and include the rows that eva

In [19]:
person.join(graduateProgram, joinExpression, "outer").show()

+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|  Purvil Dave|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|Bhavika Joshi|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|   2|   Japan Dave|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|null|         null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+



In [21]:
spark.sql("""
    SELECT * 
    FROM person
    FULL OUTER JOIN graduateProgram
    ON graduate_program = graduateProgram.id
""").show()

+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|  Purvil Dave|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|Bhavika Joshi|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|   2|   Japan Dave|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|null|         null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+



### Left Outer Join
* Keep rows with keys in the left dataset
* Left outer joins evaluate the keys in both of the DataFrames or tables and includes all rows from the left DataFrame as well as any rows in the right DataFrame that have a match in the left DataFrame.

In [22]:
graduateProgram.join(person, joinExpression,"left_outer").show()

+---+-------+--------------------+-----------+----+-------------+----------------+---------------+
| id| degree|          department|     school|  id|         name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+-------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|  Purvil Dave|               0|          [100]|
|  1|   Ph.D|                EECS|UC Berkeley|   1|Bhavika Joshi|               1|[500, 250, 100]|
|  1|   Ph.D|                EECS|UC Berkeley|   2|   Japan Dave|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|null|         null|            null|           null|
+---+-------+--------------------+-----------+----+-------------+----------------+---------------+



In [24]:
spark.sql("""
    SELECT * 
    FROM graduateProgram
    LEFT OUTER JOIN person
    ON graduate_program = graduateProgram.id
""").show()

+---+-------+--------------------+-----------+----+-------------+----------------+---------------+
| id| degree|          department|     school|  id|         name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+-------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|  Purvil Dave|               0|          [100]|
|  1|   Ph.D|                EECS|UC Berkeley|   1|Bhavika Joshi|               1|[500, 250, 100]|
|  1|   Ph.D|                EECS|UC Berkeley|   2|   Japan Dave|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|null|         null|            null|           null|
+---+-------+--------------------+-----------+----+-------------+----------------+---------------+



### Right Outer join
* Keep rows with keys in the right dataset

In [25]:
person.join(graduateProgram, joinExpression, "right_outer").show()

+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|  Purvil Dave|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|Bhavika Joshi|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|   2|   Japan Dave|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|null|         null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+



In [26]:
spark.sql("""
    SELECT *
    FROM person
    RIGHT OUTER JOIN graduateProgram
    ON person.graduate_program = graduateProgram.id
""")

DataFrame[id: bigint, name: string, graduate_program: bigint, spark_status: array<bigint>, id: bigint, degree: string, department: string, school: string]

### Left Semi Join
* Keep the rows in the left and only the left, dataset where key appears in the right dataset
* They do not actually include any values from the right DataFrame. They only compare values to see if the value exists in the second DataFrame. If the value does exist, those rows will be kept in the result, even if there are duplicate keys in the left DataFrame.

In [27]:
graduateProgram.join(person, joinExpression, "left_semi").show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [31]:
spark.sql("""
SELECT *
FROM graduateProgram 
LEFT SEMI JOIN person
ON graduateProgram.id = person.graduate_program
""").show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



### Left anti join
* Keep the rows in the left and only the left, dataset where they do not appear in the right dataset.
* Opposite of Left semi join.
* Use right frame, to compare the key and include the data of left frame wherw key does not match

In [32]:
graduateProgram.join(person, joinExpression, "left_anti").show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [34]:
spark.sql("""
SELECT * 
FROM graduateProgram
LEFT ANTI JOIN person
ON graduateProgram.id = person.graduate_program
""").show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



### Natural Join
* Perform join by implicitly matching the columns between two datasets with the same name.
* Make an implicit guess at the column on which you like to join. It finds matching columns and return the result. LEFT, RIGHT and OUTER natural join are supported.
* Dangerous when two dataframe has column with same name but the meaning underlying data is different.

```
spark.sql("""
SELECT *
FROM graduateProgram NATURAL JOIN person
""")
```

### Cross Join (Cartesian Join)
* Match every row in the left dataset with every row in the right.
* No condition.
* n row and m rows, total will be n * m rows

In [36]:
graduateProgram.crossJoin(person).show()

+---+-------+--------------------+-----------+---+-------------+----------------+---------------+
| id| degree|          department|     school| id|         name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+-------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|  Purvil Dave|               0|          [100]|
|  0|Masters|School of Informa...|UC Berkeley|  1|Bhavika Joshi|               1|[500, 250, 100]|
|  0|Masters|School of Informa...|UC Berkeley|  2|   Japan Dave|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|  0|  Purvil Dave|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|  1|Bhavika Joshi|               1|[500, 250, 100]|
|  2|Masters|                EECS|UC Berkeley|  2|   Japan Dave|               1|     [250, 100]|
|  1|   Ph.D|                EECS|UC Berkeley|  0|  Purvil Dave|               0|          [100]|
|  1|   Ph.D|       

In [39]:
spark.sql("""SELECT * FROM graduateProgram CROSS JOIN person""").show()

+---+-------+--------------------+-----------+---+-------------+----------------+---------------+
| id| degree|          department|     school| id|         name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+-------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|  Purvil Dave|               0|          [100]|
|  0|Masters|School of Informa...|UC Berkeley|  1|Bhavika Joshi|               1|[500, 250, 100]|
|  0|Masters|School of Informa...|UC Berkeley|  2|   Japan Dave|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|  0|  Purvil Dave|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|  1|Bhavika Joshi|               1|[500, 250, 100]|
|  2|Masters|                EECS|UC Berkeley|  2|   Japan Dave|               1|     [250, 100]|
|  1|   Ph.D|                EECS|UC Berkeley|  0|  Purvil Dave|               0|          [100]|
|  1|   Ph.D|       

* To enable cross join in session `spark.sql.crossJoin.enable`

* Any expression is valid join operation, as long as it returns boolean.

In [40]:
from pyspark.sql.functions import expr

In [42]:
person.withColumnRenamed("id", "personId").join(sparkStatus, expr("array_contains(spark_status, id)"),"inner").show()

+--------+-------------+----------------+---------------+---+--------------+
|personId|         name|graduate_program|   spark_status| id|        status|
+--------+-------------+----------------+---------------+---+--------------+
|       0|  Purvil Dave|               0|          [100]|100|   Contributor|
|       1|Bhavika Joshi|               1|[500, 250, 100]|500|Vice president|
|       1|Bhavika Joshi|               1|[500, 250, 100]|250|        Member|
|       1|Bhavika Joshi|               1|[500, 250, 100]|100|   Contributor|
|       2|   Japan Dave|               1|     [250, 100]|250|        Member|
|       2|   Japan Dave|               1|     [250, 100]|100|   Contributor|
+--------+-------------+----------------+---------------+---+--------------+



In [46]:
spark.sql("""
SELECT * FROM
(SELECT id as personId, name, graduate_program, spark_status 
FROM person)
INNER JOIN sparkStatus
ON array_contains(spark_status, id)
""").show()

+--------+-------------+----------------+---------------+---+--------------+
|personId|         name|graduate_program|   spark_status| id|        status|
+--------+-------------+----------------+---------------+---+--------------+
|       0|  Purvil Dave|               0|          [100]|100|   Contributor|
|       1|Bhavika Joshi|               1|[500, 250, 100]|500|Vice president|
|       1|Bhavika Joshi|               1|[500, 250, 100]|250|        Member|
|       1|Bhavika Joshi|               1|[500, 250, 100]|100|   Contributor|
|       2|   Japan Dave|               1|     [250, 100]|250|        Member|
|       2|   Japan Dave|               1|     [250, 100]|100|   Contributor|
+--------+-------------+----------------+---------------+---+--------------+



### Handling Duplicate Column Names

* Different join expression
    - Change join expression from boolean to string will removes one of the columns for you during the join.

In [50]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")

In [52]:
gradProgramDupe.show()

+----------------+-------+--------------------+-----------+
|graduate_program| degree|          department|     school|
+----------------+-------+--------------------+-----------+
|               0|Masters|School of Informa...|UC Berkeley|
|               2|Masters|                EECS|UC Berkeley|
|               1|   Ph.D|                EECS|UC Berkeley|
+----------------+-------+--------------------+-----------+



In [53]:
person.show()

+---+-------------+----------------+---------------+
| id|         name|graduate_program|   spark_status|
+---+-------------+----------------+---------------+
|  0|  Purvil Dave|               0|          [100]|
|  1|Bhavika Joshi|               1|[500, 250, 100]|
|  2|   Japan Dave|               1|     [250, 100]|
+---+-------------+----------------+---------------+



In [55]:
person.join(gradProgramDupe, "graduate_program").show()

+----------------+---+-------------+---------------+-------+--------------------+-----------+
|graduate_program| id|         name|   spark_status| degree|          department|     school|
+----------------+---+-------------+---------------+-------+--------------------+-----------+
|               0|  0|  Purvil Dave|          [100]|Masters|School of Informa...|UC Berkeley|
|               1|  1|Bhavika Joshi|[500, 250, 100]|   Ph.D|                EECS|UC Berkeley|
|               1|  2|   Japan Dave|     [250, 100]|   Ph.D|                EECS|UC Berkeley|
+----------------+---+-------------+---------------+-------+--------------------+-----------+



In [56]:
person.join(gradProgramDupe, person["graduate_program"] == gradProgramDupe["graduate_program"]).show()

+---+-------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+-------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|  Purvil Dave|               0|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|Bhavika Joshi|               1|[500, 250, 100]|               1|   Ph.D|                EECS|UC Berkeley|
|  2|   Japan Dave|               1|     [250, 100]|               1|   Ph.D|                EECS|UC Berkeley|
+---+-------------+----------------+---------------+----------------+-------+--------------------+-----------+



* Drop column after join

In [63]:
from pyspark.sql.functions import col

In [65]:
person.join(gradProgramDupe, person["graduate_program"] == gradProgramDupe["graduate_program"]).drop(person["graduate_program"]).show()

+---+-------------+---------------+----------------+-------+--------------------+-----------+
| id|         name|   spark_status|graduate_program| degree|          department|     school|
+---+-------------+---------------+----------------+-------+--------------------+-----------+
|  0|  Purvil Dave|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|Bhavika Joshi|[500, 250, 100]|               1|   Ph.D|                EECS|UC Berkeley|
|  2|   Japan Dave|     [250, 100]|               1|   Ph.D|                EECS|UC Berkeley|
+---+-------------+---------------+----------------+-------+--------------------+-----------+



* Rename ecolumn before join

### How Spark performs join?

* Spark do cluster communication
    - Shuffle join, all to all communication, 
    - broadcast join
* In spark we have big table and small table
* Big table to big table
    - Shuffle join takes place, every node will talk to other node, they share data according to which node has certain keys. Expensive joins because network has lots of traffic if data is not partitioned well.
    ![](images/big_to_big.png)

* Big table to small table
    - When table is small enough to fit in memory of single worker node, We can use broadcast join.
    - Replicate small dataframe to all worker node in beginning. So here we do all overhead at beginning so, individual worker node can work without waiting or communicating with other worker node.
    ![](images/broadcast_join.png)

In [66]:
person.join(graduateProgram, joinExpression).explain()

== Physical Plan ==
*(5) SortMergeJoin [graduate_program#10L], [id#24L], Inner
:- *(2) Sort [graduate_program#10L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(graduate_program#10L, 200)
:     +- *(1) Project [_1#0L AS id#8L, _2#1 AS name#9, _3#2L AS graduate_program#10L, _4#3 AS spark_status#11]
:        +- *(1) Filter isnotnull(_3#2L)
:           +- Scan ExistingRDD[_1#0L,_2#1,_3#2L,_4#3]
+- *(4) Sort [id#24L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(id#24L, 200)
      +- *(3) Project [_1#16L AS id#24L, _2#17 AS degree#25, _3#18 AS department#26, _4#19 AS school#27]
         +- *(3) Filter isnotnull(_1#16L)
            +- Scan ExistingRDD[_1#16L,_2#17,_3#18,_4#19]


In [67]:
* Even we can hint optimizer that we like to perform certain kind of join

SyntaxError: invalid syntax (<ipython-input-67-d5a4580bd9a8>, line 1)

In [68]:
from pyspark.sql.functions import broadcast

In [69]:
person.join(broadcast(graduateProgram), joinExpression).explain()

== Physical Plan ==
*(2) BroadcastHashJoin [graduate_program#10L], [id#24L], Inner, BuildRight
:- *(2) Project [_1#0L AS id#8L, _2#1 AS name#9, _3#2L AS graduate_program#10L, _4#3 AS spark_status#11]
:  +- *(2) Filter isnotnull(_3#2L)
:     +- Scan ExistingRDD[_1#0L,_2#1,_3#2L,_4#3]
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true]))
   +- *(1) Project [_1#16L AS id#24L, _2#17 AS degree#25, _3#18 AS department#26, _4#19 AS school#27]
      +- *(1) Filter isnotnull(_1#16L)
         +- Scan ExistingRDD[_1#16L,_2#17,_3#18,_4#19]
